In [2]:
# import keras libs 
from keras import models # model module 
from keras.layers import LSTM, Dense, Activation, Dropout  # layers for Sequential models 
from keras.models import Sequential # sequential model 
from keras.preprocessing.sequence import sequence # import sequence
from keras.callbacks import TensorBoard # for tensorboard viz 
from keras.optimizers import Adadelta, RMSprop

# load helper libraries 
from mido import MidiFile, MidiTrack, Message # to handdle mido file 
import mido 
import numpy as np # for matrix maths 

import musicmido as mm # to preprocess mido files

Using TensorFlow backend.


In [3]:
tensorboard = TensorBoard(log_dir='./logs/pav0000ecd')

In [4]:
mid_f = mm.getMusicFile(file_path='./Pavana.mid')

mid_f

<midi file './Pavana.mid' type 1, 10 tracks, 1254 messages>

In [5]:
# helper function for notes extraction
notes = [] # empty notes list
# intial params 
time = float(0)
prev = float(0)

for msg in mid_f:
    time += msg.time # to get time of note 
    # only get music data not meta data 
    if not msg.is_meta:
        # only intrested in piano channel 
        if msg.channel in [0,1,2,3,4,9]:
            if msg.type == 'note_on':
                note = msg.bytes() # get notes data in form of bytes 
                note = note[1:3] # only get note and velocity of our model 
                note.append(time - prev)
                prev = time # set prev to current time 
                notes.append(note)

notes # return list of notes extracted

[[87, 127, 0.0],
 [87, 0, 0.5393256],
 [87, 127, 0.13483139999999993],
 [87, 0, 0.2696628],
 [87, 127, 0.06741570000000008],
 [87, 0, 0.2696628000000001],
 [87, 127, 0.06741569999999997],
 [87, 0, 0.5393256],
 [87, 127, 0.13483139999999993],
 [87, 0, 0.26966279999999987],
 [87, 127, 0.06741570000000019],
 [87, 0, 0.26966279999999987],
 [87, 127, 0.06741570000000019],
 [87, 0, 0.5393256000000002],
 [87, 127, 0.13483139999999993],
 [87, 0, 0.5393256000000002],
 [81, 127, 0.13483140000000038],
 [76, 127, 0.0],
 [57, 127, 0.0],
 [57, 127, 0.0],
 [87, 127, 0.0],
 [87, 0, 0.5393255999999997],
 [87, 127, 0.13483140000000038],
 [87, 0, 0.26966279999999987],
 [87, 127, 0.06741569999999975],
 [81, 0, 0.06741569999999975],
 [76, 0, 0.0],
 [57, 0, 0.0],
 [57, 0, 0.0],
 [87, 0, 0.20224710000000012],
 [81, 127, 0.06741569999999975],
 [76, 127, 0.0],
 [60, 127, 0.0],
 [57, 127, 0.0],
 [87, 127, 0.0],
 [81, 0, 0.5393255999999997],
 [76, 0, 0.0],
 [60, 0, 0.0],
 [57, 0, 0.0],
 [87, 0, 0.0],
 [80, 127, 

In [6]:
t = []
for note in notes:
    note[0] = (note[0]-24)/ 88 # scale 1d of notes
    note[1] = (note[1]) / 127 # scale 2d of notes 
    t.append(note[2])
max_t = max(t) # max of note[2]
for note in notes:
    note[2] = note[2] / max_t # scale note[2]

# create data, labels for predictions 
X = []
Y = []
n_prev = 30 # num of notes to predict 
for i in range(len(notes) - n_prev):
    x = notes[i:i+n_prev] # value of notes as features
    y = notes[i+n_prev] # labels of notes 
    X.append(x) # make a dict of x 
    Y.append(y) # make list of y

# save a seed to do predictions
seed = notes[0:n_prev]

# make arrays of labels and prediction for better operations 
X = np.array(X) 
Y = np.array(Y)


# print values of x and y 
print("Size of X {}".format(X.shape))
print("Size of Y {}".format(Y.shape))

# print data values 
print("Starting 5 values of X {}".format(X[:5]))
print("Starting 5 values of Y {}".format(Y[:5]))

Size of X (1174, 30, 3)
Size of Y (1174, 3)
Starting 5 values of X [[[0.        1.        0.       ]
  [0.        0.        0.898876 ]
  [0.        1.        0.224719 ]
  [0.        0.        0.449438 ]
  [0.        1.        0.1123595]
  [0.        0.        0.449438 ]
  [0.        1.        0.1123595]
  [0.        0.        0.898876 ]
  [0.        1.        0.224719 ]
  [0.        0.        0.449438 ]
  [0.        1.        0.1123595]
  [0.        0.        0.449438 ]
  [0.        1.        0.1123595]
  [0.        0.        0.898876 ]
  [0.        1.        0.224719 ]
  [0.        0.        0.898876 ]
  [0.        1.        0.224719 ]
  [0.        1.        0.       ]
  [0.        1.        0.       ]
  [0.        1.        0.       ]
  [0.        1.        0.       ]
  [0.        0.        0.898876 ]
  [0.        1.        0.224719 ]
  [0.        0.        0.449438 ]
  [0.        1.        0.1123595]
  [0.        0.        0.1123595]
  [0.        0.        0.       ]
  [0.        0.

In [19]:
# make a keras model for music generation 
print "Building Model....."
model = Sequential() # initialize a sequential model 
# model.add(LSTM(512,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
# model.add(Dropout(0.75)) # 1st layer of Dropout 
# model.add(LSTM(512,return_sequences=True, input_shape=(n_prev, 3))) # 2nd layer of LSTM 
# model.add(Dropout(0.75)) # 2nd layer of Dropout
# model.add(LSTM(256,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
# model.add(Dropout(0.75)) # 1st layer of Dropout 
# model.add(LSTM(256,return_sequences=True, input_shape=(n_prev, 3))) # 2nd layer of LSTM 
# model.add(Dropout(0.75)) # 2nd layer of Dropout
# model.add(LSTM(128,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
# model.add(Dropout(0.75)) # 1st layer of Dropout 
# model.add(LSTM(64,return_sequences=True, input_shape=(n_prev, 3))) # 2nd layer of LSTM 
# model.add(Dropout(0.75)) # 2nd layer of Dropout
# model.add(LSTM(32,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
# model.add(Dropout(0.75)) # 1st layer of Dropout 
# model.add(LSTM(64,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
# model.add(Dropout(0.75)) # 1st layer of Dropout 
# model.add(LSTM(64,return_sequences=True, input_shape=(n_prev, 3))) # 2nd layer of LSTM 
# model.add(Dropout(0.75)) # 2nd layer of Dropout
# model.add(LSTM(64,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
# model.add(Dropout(0.75)) # 1st layer of Dropout 
# model.add(LSTM(32,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
# model.add(Dropout(0.75)) # 1st layer of Dropout 
# model.add(LSTM(64,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
# model.add(Dropout(0.75)) # 1st layer of Dropout 
model.add(LSTM(512,return_sequences=False, input_shape=(n_prev, 3))) # 1st layer of LSTM 
model.add(Dropout(0.75)) # 1st layer of Dropout 
# model.add(LSTM(64,return_sequences=False, input_shape=(n_prev, 3))) # 2nd layer of LSTM 
# model.add(Dropout(0.75)) # 2nd layer of Dropout
model.add(Dense(3)) # a dense layer of 3 tensors 


adadel = Adadelta()
rms = RMSprop(lr=1e-4)
model.add(Activation('linear'))

# compile model
print("Compiling Model......")
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
print("Model compiled sucessfully")

Building Model.....
Compiling Model......
Model compiled sucessfully


In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 512)               1056768   
_________________________________________________________________
dropout_21 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 1539      
_________________________________________________________________
activation_5 (Activation)    (None, 3)                 0         
Total params: 1,058,307
Trainable params: 1,058,307
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(X, Y, batch_size=300, epochs=50, callbacks=[tensorboard])

Epoch 1/50
1174/1174 [==============================] - 7s 6ms/step - loss: 0.1677 - acc: 0.4063
Epoch 2/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.1230 - acc: 0.4991
Epoch 3/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.1058 - acc: 0.5000
Epoch 4/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.1080 - acc: 0.5068
Epoch 5/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.0863 - acc: 0.5017
Epoch 6/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.0709 - acc: 0.5102
Epoch 7/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.0702 - acc: 0.5213
Epoch 8/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.0761 - acc: 0.5264
Epoch 9/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.0704 - acc: 0.5128
Epoch 10/50
1174/1174 [==============================] - 6s 5ms/step - loss: 0.0630 - acc: 0.5102
Epoch 11/50
1174/1174 [======

In [22]:
model.evaluate(X,Y, batch_size=300)

1174/1174 [==============================] - 3s 2ms/step


[0.03765918827874291, 0.74275979749998]

In [ ]:
# make predictions 
predictions = []
x = np.array(seed)
x = np.expand_dims(x, axis=0)

for i in range(3000):
    preds = model.predict(x)
    x = np.squeeze(x)
    x = np.concatenate((x, preds))
    x = x[1:]
    x = np.expand_dims(x, axis=0)
    preds = np.squeeze(preds)
    predictions.append(preds)

In [ ]:
# filter predictions and remove outliers
import random
for pred in predictions:
    pred[0] = int(88*pred[0] + 24)
    pred[1] = int(127*pred[1])
    pred[2] *= max_t
    
    # reject values that would be out of the range 
    if pred[0] < 24:
        pred[0] = random.randint(24, 50)
    elif pred[0] > 102:
        pred[0] = random.randint(80, 102)
    if pred[1] < 0:
        pred[1] = random.randint(0, 20)
    elif pred[1] > 127:
        pred[1] = random.randint(100, 127)
    if pred[2] < 0:
        pred[2] = 0

In [ ]:
# save track into mid file 
mid = MidiFile()
track = MidiTrack()

mid.tracks.append(track)

for note in predictions:
    note = np.insert(note, 0, 147)
    bytes = note.astype(int)
    print(note)
    msg = Message.from_bytes(bytes[0:3])
    time = int(note[3] / 0.001025) # to rescale to midi's delta ticks. arbitary for now 
    msg.time = time 
    track.append(msg)
    
mid.save('new_pav_0_4.mid') # save song into a track

In [ ]:
model.get_layer

In [ ]:
model.history.history.keys()

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
plt.plot(model.history.history['loss'])

In [ ]:
from keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs/pav04')

In [ ]:
model.fit(X, Y, epochs=50, batch_size=300, callbacks=[tensorboard])